In [6]:
# Libraries
import numpy as np
from matplotlib import pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
import copy 
import pandas as pd
from scipy.sparse import csr_matrix, lil_matrix
from tqdm.notebook import tnrange, tqdm
from sklearn import metrics
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

In [7]:
## Helper functions
# Check if all elements of array are the same
def all_same(a):
    return len(set(a)) == 1

In [8]:
## Load data
data = pd.read_csv('diversion_pp.csv', names=['neigh', 'y', 'zips', 'id', 'dist', 'z'], 
                   header=0, usecols=[1, 2, 3, 4, 5, 6], low_memory=False)
zip_data = pd.read_csv('zip_dta.csv', names=['neigh', 'lon', 'lat', 'zips', 'y', 'closest_int'],
                       header=0, usecols=[1, 2, 3, 4, 5, 6])
pp_data = pd.read_csv('pp_dta.csv').rename(columns={'Trt': 'z'})
pp_data = pp_data.drop(pp_data.columns[0], axis=1)

In [9]:
## Power plant data processing
num_clusters = len(np.unique(pp_data['neigh']))
num_pp = pp_data.groupby('neigh').agg({'neigh': 'count'}).rename(columns={'neigh': 'num_pp'})
num_pp.index.names = ['cluster']
pp_data = pp_data.join(num_pp, on='cluster').drop(columns=['neigh'])

In [10]:
## Estimate a GPS model
def gps_estimation(pp_data, method='svm'):
    clusters = np.unique(pp_data['cluster'])
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(clusters.reshape(-1, 1))
    cluster_one_hot = pd.DataFrame(enc.transform(pp_data.cluster.values.reshape(-1, 1)).toarray())
    pp_data_temp = pd.concat([pp_data.drop(columns=['cluster']), cluster_one_hot], axis=1)
    
    X = pp_data_temp.drop(columns=['z', 'id'])
    y = pp_data_temp.z
    
    if method == 'svm':
        param_grid = {'C': [10e0, 5e0, 2e0, 1e0, 1e-1, 1e-2, 1e-3], 'kernel': ['rbf', 'linear']}
        mod_svm = GridSearchCV(SVC(probability=True), param_grid=param_grid)
        mod_svm.fit(X, y)
        gps = mod_svm.predict_proba(X)[:,1].reshape(-1, 1)
    elif method == 'forest':
        # Number of trees in random forest
        n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
        # Number of features to consider at every split
        max_features = ['auto', 'sqrt']
        # Maximum number of levels in tree
        max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
        max_depth.append(None)
        # Minimum number of samples required to split a node
        min_samples_split = [2, 5, 10]
        # Minimum number of samples required at each leaf node
        min_samples_leaf = [1, 2, 4]
        # Method of selecting samples for training each tree
        bootstrap = [True, False]
        # Create the random grid
        random_grid = {'n_estimators': n_estimators,
                       'max_features': max_features,
                       'max_depth': max_depth,
                       'min_samples_split': min_samples_split,
                       'min_samples_leaf': min_samples_leaf,
                       'bootstrap': bootstrap}
        mod_forest = RandomizedSearchCV(RandomForestClassifier(random_state=666, class_weight='balanced'),
                                        param_distributions=random_grid, n_iter=100, cv=5, 
                                        random_state=666, n_jobs=-1)
        mod_forest.fit(X, y)
        gps = mod_forest.predict_proba(X)[:,1].reshape(-1, 1)
    else:
        print('Incorrect prediciton method: {}'.format(method))
        return
        
    return pp_data.assign(gps=gps)

In [11]:
pp_data = gps_estimation(pp_data)

In [12]:
## Add a gps column to data
def merge_gps_data(data, pp_data):
    return data.join(pp_data.gps, on='id')

In [13]:
data = merge_gps_data(data, pp_data)

In [14]:
## Compute exposures given the assignment
def get_exposure(data, trt=None):
    all_zips = np.unique(data['zips'])
    d = {'zips': all_zips, 'e': 0.0, 'p': 0.0}
    zips_with_e = pd.DataFrame(d).set_index('zips')
    
    if trt is not None:
        data = data.drop(['z'], axis=1) \
            .join(trt, on='id', how='inner')
    
    for i in all_zips:
        data_i = data.loc[data.zips == i]
        #print(np.prod((data_i.gps ** data_i.z) * ((1 - data_i.gps) ** (1 - data_i.z))))
        zips_with_e.loc[i].e = np.sum(data_i.dist * data_i.z) / np.sum(data_i.dist)
        zips_with_e.loc[i].p = np.prod((data_i.gps ** data_i.z) * ((1 - data_i.gps) ** (1 - data_i.z)))

    return zips_with_e

In [15]:
zips_with_e = get_exposure(data)

In [16]:
all_ids = np.unique(data['id'])
trt_0 = pd.DataFrame({'all_ids': all_ids, 'z': 0.0})
trt_1 = pd.DataFrame({'all_ids': all_ids, 'z': 1.0})
zips_with_0 = get_exposure(data, trt_0).rename(columns={'e': 'e_0', 'p': 'p_0'})
zips_with_1 = get_exposure(data, trt_1).rename(columns={'e': 'e_1', 'p': 'p_1'})

In [17]:
dataset_final = zip_data.join(zips_with_e, on='zips') \
    .join(zips_with_0, on='zips') \
    .join(zips_with_1, on='zips')

In [18]:
## Variables
def generate_vars(dataset_final):
    y = dataset_final.y.to_numpy().reshape((-1,1))
    e = dataset_final.e.to_numpy().reshape((-1,1))
    gps = dataset_final.p.to_numpy().reshape((-1,1))
    gps_at_0 = dataset_final.p_0.to_numpy().reshape((-1,1))
    gps_at_1 = dataset_final.p_1.to_numpy().reshape((-1,1))
    gps_at = np.column_stack((gps_at_0, gps_at_1))
    return (y, e, gps, gps_at)

In [19]:
## Function that estimates a naive model
def est_naive(y, e):
    # Matrix of regressors
    X = sm.add_constant(e)
    # Regress y on exposure and constant
    model = sm.OLS(y, X).fit()
    params_hat = model.params.reshape((-1,1))
    eps_hat = y - model.predict().reshape((-1,1))
    return (params_hat, eps_hat)

def ate_naive(y, e):
    return est_naive(y, e)[0][1]

## Function that estimates a kernell ridge regression
def ate_kernel(y, e, gps, gps_at):
    # Sample size
    N = y.size
    # Cross-validate the model
    param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3]}
    kr = GridSearchCV(KernelRidge(), param_grid=param_grid)
    X = np.column_stack((e, gps))
    kr.fit(X, y)
    # Get the predictions
    X_0 = np.column_stack((np.zeros((N,1)), gps_at[:,0]))
    y_0 = kr.predict(X_0)
    X_1 = np.column_stack((np.ones((N,1)), gps_at[:,-1]))
    y_1 = kr.predict(X_1)
    return np.mean(y_1 - y_0)

In [20]:
## Bootstrap CIs
def compute_bootstrap_CI(y, boot_iter_func, est_func, *args, num_boot_iter=1000, alpha=0.05):
    # Check if all elements are the same
    # Bootstrap iterations
    beta_hat_boot_dist_unsorted = np.zeros(num_boot_iter)
    for b in tnrange(num_boot_iter, desc='Bootstrap'): #tqdm_notebook(range(num_boot_iter), desc='Bootsrap:'):
        beta_hat_boot_dist_unsorted[b] = boot_iter_func(y, est_func, *args)
    beta_hat_boot_dist = np.sort(beta_hat_boot_dist_unsorted)
    # Sample observations
    q_lo = beta_hat_boot_dist[int(np.floor(num_boot_iter * alpha / 2.0))]
    q_hi = beta_hat_boot_dist[min(int(np.ceil(num_boot_iter * (1.0 - alpha / 2.0))), 
                                  num_boot_iter - 1)] # check for out-of-bounds
    return (q_lo, q_hi)

## Naive bootstrap iteration
def bootstrap_naive_iter(y, est_func, e, *args):
    # Sample size
    N = y.size
    while True:
        # Sample
        boot_sample = np.random.choice(N, N, replace=True)
        # Bootstrap data
        y_b = y[boot_sample,0] # Bootstrap outcomes
        e_b = e[boot_sample,0] # Bootstrap exposures
        if not all_same(e_b):
            break
    args_b = (a if callable(a) else a[boot_sample,:] for a in args)
    return est_func(y_b, e_b, *args_b)

In [21]:
def main(data, num_boot_iter=1000, alpha=0.05, seed=666):
    # Set seed
    np.random.seed(seed)
    # Get data
    (y, e, gps, gps_at) = generate_vars(data)
    # Naive estimate
    ate_naive_val = ate_naive(y, e)
    # Kernel Ridge estimate
    ate_kernel_val = ate_kernel(y, e, gps, gps_at)
    # Naive bootstrap naive estimate
    (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_naive_iter, ate_naive, e, num_boot_iter=num_boot_iter, alpha=alpha)
    ci_naive_naive = [q_lo, q_hi]
    # Naive bootstrap kernel estimate
    (q_lo, q_hi) = compute_bootstrap_CI(y, bootstrap_naive_iter, ate_kernel, e, gps, gps_at, num_boot_iter=num_boot_iter, alpha=alpha)
    ci_naive_kernel = [q_lo, q_hi]
    # Print the results
    print('Naive estimate: {}. Kernel ridge estimate: {}.' \
         .format(ate_naive_val, ate_kernel_val))
    print('Naive CI: [{},{}]. Kernel ridge CI: [{},{}].' \
         .format(ci_naive_naive[0], ci_naive_naive[1], ci_naive_kernel[0], ci_naive_kernel[1]))

In [ ]:
## Run the main function
main(dataset_final, num_boot_iter=1000, alpha=0.05, seed=666)